In [3]:
import networkx as nx
from collections import defaultdict

# >>> ajuste aqui <<<
gexf_path = "../data/networks/coauthorship-network-2019_2020_2021_2022-20251207-061133.gexf"

# carregar rede
G = nx.read_gexf(gexf_path)

print(f"N nós: {G.number_of_nodes()}")
print(f"N arestas: {G.number_of_edges()}")

# tentar detectar nome do atributo do partido
party_attr_name = None
for _, data in G.nodes(data=True):
    for candidate in ["party", "siglaPartido", "partido"]:
        if candidate in data:
            party_attr_name = candidate
            break
    if party_attr_name:
        break

print(f"Atributo de partido detectado: {party_attr_name}")

# mapear nós -> partido (ou None)
party_of = {}
for n, data in G.nodes(data=True):
    partido = data.get(party_attr_name)
    party_of[n] = partido

# acumular peso por partido
peso_por_partido = defaultdict(float)

for u, v, data in G.edges(data=True):
    w = data.get("weight", 1)

    pu = party_of.get(u)
    pv = party_of.get(v)

    # conte para ambos os lados, ignorando nós sem partido
    if pu:
        peso_por_partido[pu] += w
    if pv:
        peso_por_partido[pv] += w

# ordenar do maior para o menor
resultado = sorted(peso_por_partido.items(), key=lambda x: x[1], reverse=True)

print("\n=== PESO TOTAL POR PARTIDO ===")
for partido, peso in resultado:
    print(f"{partido:>6}  {peso:,.0f}")



N nós: 336
N arestas: 40398
Atributo de partido detectado: party

=== PESO TOTAL POR PARTIDO ===
    PT  178,917
    PL  29,980
    PP  21,809
 UNIÃO  21,058
   PSD  20,400
REPUBLICANOS  19,603
   MDB  13,915
 PCdoB  9,563
  PSOL  8,646
   PDT  8,039
   PSB  6,792
  PSDB  4,168
CIDADANIA  2,924
  PODE  2,479
SOLIDARIEDADE  2,366
AVANTE  2,132
   PRD  1,688
  NOVO  1,322
    PV  1,304
  REDE  1,115
S.PART.  362


In [5]:
from collections import defaultdict
import pandas as pd
import networkx as nx

# Se G ainda não estiver carregado nesta sessão, descomente:
# gexf_path = "../data/networks/coauthorship-network-2019_2020_2021_2022.gexf"
# G = nx.read_gexf(gexf_path)

print(f"N nós: {G.number_of_nodes()}, N arestas: {G.number_of_edges()}")

# 1) Detectar automaticamente o atributo de partido
party_attr_name = None
for _, data in G.nodes(data=True):
    for candidate in ["party", "siglaPartido", "partido"]:
        if candidate in data:
            party_attr_name = candidate
            break
    if party_attr_name:
        break

print("Atributo de partido detectado:", party_attr_name)

# 2) Mapear nós -> partido
party_of = {}
for n, data in G.nodes(data=True):
    partido = data.get(party_attr_name)
    party_of[n] = partido

# 3) Tamanho de cada partido (número de nós/deputados na rede)
nodes_by_party = defaultdict(set)
for n, p in party_of.items():
    if p:
        nodes_by_party[p].add(n)

party_sizes = {p: len(ns) for p, ns in nodes_by_party.items()}

# 4) Coautoria partido–partido (ponderada pelo weight da aresta)
#    Vamos construir uma matriz DIRECIONADA:
#    A->B = quanto do peso de coautoria "passa" de A para B
weight_by_party_pair = defaultdict(float)

for u, v, data in G.edges(data=True):
    w = data.get("weight", 1.0)

    pu = party_of.get(u)
    pv = party_of.get(v)

    # ignorar arestas onde algum lado não tem partido
    if pu is None or pv is None:
        continue

    # contar nos dois sentidos para poder olhar "do ponto de vista de A"
    weight_by_party_pair[(pu, pv)] += w
    if pu != pv:
        weight_by_party_pair[(pv, pu)] += w

# 5) Total de coautoria de cada partido A (somando A->qualquer B)
total_by_party = defaultdict(float)
for (pa, pb), w in weight_by_party_pair.items():
    total_by_party[pa] += w

# 6) DataFrame com matriz A coautora com B (peso e fração)
rows = []
for (pa, pb), w in weight_by_party_pair.items():
    total_pa = total_by_party[pa]
    frac = w / total_pa if total_pa > 0 else 0.0
    rows.append({
        "partido_A": pa,
        "partido_B": pb,
        "peso_AB": w,
        "frac_coautoria_de_A_para_B": frac
    })

df_party_party = pd.DataFrame(rows)

# 7) Para cada partido_A, encontrar o B com maior fração de coautoria
melhor_parceria = []

for partido_A in sorted(party_sizes.keys()):
    sub = df_party_party[df_party_party["partido_A"] == partido_A]
    if sub.empty:
        continue
    sub = sub.sort_values(
        by=["frac_coautoria_de_A_para_B", "peso_AB"],
        ascending=False
    )
    top = sub.iloc[0]
    melhor_parceria.append({
        "partido_A": partido_A,
        "tam_bancada_A": party_sizes.get(partido_A, 0),
        "partido_B_top": top["partido_B"],
        "peso_AB": top["peso_AB"],
        "frac_coautoria_de_A_para_B": top["frac_coautoria_de_A_para_B"]
    })

df_top_parceria = pd.DataFrame(melhor_parceria).sort_values(
    by="frac_coautoria_de_A_para_B",
    ascending=False
)

df_top_parceria.reset_index(drop=True, inplace=True)

print("Ranking: para cada partido A, qual B é o parceiro preferencial (normalizado pela coautoria total de A):")
df_top_parceria


N nós: 336, N arestas: 40398
Atributo de partido detectado: party
Ranking: para cada partido A, qual B é o parceiro preferencial (normalizado pela coautoria total de A):


,partido_A,tam_bancada_A,partido_B_top,peso_AB,frac_coautoria_de_A_para_B
0,PT,46,PT,71888.0,0.671668
1,PSOL,7,PT,3788.0,0.494840
2,REDE,1,PT,542.0,0.486099
3,PCdoB,7,PT,3533.0,0.387008
4,PV,2,PT,420.0,0.322829
5,PSB,9,PT,1903.0,0.289254
6,PDT,11,PT,2161.0,0.277122
7,PSDB,10,PT,947.0,0.230022
8,SOLIDARIEDADE,4,PT,521.0,0.221231
9,CIDADANIA,5,PT,631.0,0.217361


In [7]:
import networkx as nx

def calc_total_weight(path1: str, path2: str):
    def total_weight(g):
        w = 0
        for _, _, data in g.edges(data=True):
            w += float(data.get("weight", 0))
        return w

    print("Carregando rede 1:", path1)
    G1 = nx.read_gexf(path1)

    print("Carregando rede 2:", path2)
    G2 = nx.read_gexf(path2)

    tw1 = total_weight(G1)
    tw2 = total_weight(G2)

    print("\n===== RESULTADO =====")
    print(f"Rede 1: peso total = {tw1}")
    print(f"Rede 2: peso total = {tw2}")
    print("=====================\n")

    return tw1, tw2

In [13]:
calc_total_weight("../data/networks/covoting-2019_2020_2021_2022-20251208-051415.gexf","../data/networks/coauthorship-network-2019_2020_2021_2022-20251208-051609.gexf")

Carregando rede 1: ../data/networks/covoting-2019_2020_2021_2022-20251208-051415.gexf
Carregando rede 2: ../data/networks/coauthorship-network-2019_2020_2021_2022-20251208-051609.gexf

===== RESULTADO =====
Rede 1: peso total = 5012979.0
Rede 2: peso total = 179291.0



(5012979.0, 179291.0)